In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

# Dev_users 가져오기

In [3]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    #print(len(lines))
    #print(type(lines))
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
#dev_users_list[:2]

In [2]:
test_users_path = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    #print(len(lines))
    #print(type(lines))
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')
#test_users_list[:2]

# Read Raw 만들기

In [4]:
## 사전에 필요한 data load

users = pd.read_json(directory + 'users.json', lines=True)
magazine = pd.read_json(directory + 'magazine.json', lines=True)
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [5]:
atc = metadata.copy()
atc['reg_datetime'] = atc['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
atc.loc[atc['reg_datetime'] == atc['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
atc['reg_dt'] = atc['reg_datetime'].dt.date
atc['type'] = atc['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')
# 컬럼명 변경
atc.columns = ['id', 'display_url', 'article_id', 'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title', 'author_id', 'reg_datetime', 'reg_dt', 'type']
atc.head()
atc_cnt_by_reg_dt = atc.groupby('reg_dt', as_index=False)['article_id'].count()

In [6]:
read_file_lst = glob.glob(directory + 'read/read/*')
exclude_file_lst = ['read.tar']
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)
        
read = pd.concat(read_df_lst)
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))
read_cnt_by_user = read['article_id'].str.split(' ').map(len)
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})
read_raw.shape

(22110706, 4)

In [7]:
atc_read_cnt = read_raw[read_raw.article_id != ''].groupby('article_id')['user_id'].count()
atc_read_cnt = atc_read_cnt.reset_index()
atc_read_cnt.columns = ['article_id', 'read_cnt']
#metadata 결합
atc_read_cnt = pd.merge(atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')
# metadata를 찾을 수 없는 소비 로그 제외
atc_read_cnt_nn = atc_read_cnt[atc_read_cnt['id'].notnull()]
# 소비수 기준 분류값
def get_class(x):
    if x >= 142:
        result = '5%'
    elif x >= 72:
        result = '10%'
    elif x >= 25:
        result = '25%'
    elif x >= 8:
        result = '50%'
    elif x >= 3:
        result = '75%'
    else:
        result = '100%'
    return result

atc_read_cnt_nn['class'] = atc_read_cnt_nn['read_cnt'].map(get_class)
#atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False).head(2)

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
off_data = pd.merge(read_raw, atc, how='inner', left_on='article_id', right_on='article_id')
off_data.columns = ['read_dt', 'hr', 'user_id', 'article_id', 'article_seq', 'display_url',
                    'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title',
                    'author_id', 'reg_datetime', 'reg_dt', 'type']
off_data = off_data[['read_dt', 'user_id', 'article_id', 'title', 'sub_title', 'author_id', 'reg_dt', 'type', 'display_url', 'keyword_list', 'magazine_id']]
off_data['read_dt'] = pd.to_datetime(off_data['read_dt'], format='%Y%m%d')
off_data['reg_dt'] = pd.to_datetime(off_data['reg_dt'], format='%Y-%m-%d')
off_data['off_day'] = (off_data['read_dt'] - off_data['reg_dt']).dt.days
# meatadata와 join
off_data = pd.merge(off_data, atc_read_cnt_nn[['article_id', 'read_cnt', 'class']], how='left', left_on='article_id', right_on='article_id')
off_data.shape

(20905040, 14)

In [23]:
#del atc, atc_read_cnt, metadata, atc_cnt_by_reg_dt, read, read_raw

# Total Users

In [9]:
grp_by_user = off_data.groupby('user_id').agg({'read_dt':['nunique', 'count']})
grp_by_user = grp_by_user.reset_index()
grp_by_user.columns = ['user_id', 'visit_day_cnt', 'read_cnt']
#grp_by_user.head(3)
#grp_by_user.shape #(303047, 3)

In [10]:
#list(grp_by_user[1:2]['visit_day_cnt'])[0]
#k = grp_by_user[0:1]
#print(type(k))

In [46]:
cnt = 0
dev_user_id = []
dev_visit_day_cnt = []
dev_read_cnt = []
# 아 근데 read raw로 해야하지 않을까? 생각보다 많은 글들이 사라진 것 같은데
for i in range(len(grp_by_user['user_id'])):
    if list(grp_by_user[i:i+1]['user_id'])[0] in dev_users_list:
        dev_user_id.append(list(grp_by_user[i:i+1]['user_id'])[0])
        dev_visit_day_cnt.append(list(grp_by_user[i:i+1]['visit_day_cnt'])[0])
        dev_read_cnt.append(list(grp_by_user[i:i+1]['read_cnt'])[0])
        cnt+=1

In [48]:
for x in dev_users_list:
    if x not in dev_user_id:
        dev_user_id.append(x)
        dev_visit_day_cnt.append(0)
        dev_read_cnt.append(0)
        cnt+=1

In [55]:
print(len(dev_user_id))
print(len(dev_visit_day_cnt))
print(len(dev_read_cnt))

# check 
error = 0
for x in dev_user_id :
    if x not in dev_users_list:
        print(x)
        error +=1

3000
3000
3000


In [57]:
dev_user_id[0]
read_article = off_data[off_data_author_user['user_id'] == dev_user_id[0]]['article_id']
list(set(read_article))
#len(dev_read_article_list)

'#000fe61478d384d09f3bcdd0c2f5227d'

In [88]:
print(list(off_data[1:2]['read_dt'])[0])
print(list(off_data[3:4]['read_dt'])[0])
list(off_data[1:2]['read_dt'])[0] >= list(off_data[3:4]['read_dt'])[0]
type(list(off_data[3:4]['read_dt'])[0])

2018-10-01 00:00:00
2018-10-01 00:00:00


pandas._libs.tslibs.timestamps.Timestamp

In [69]:
a = 20190222
a = pd.to_datetime(a,format='%Y%m%d')
a

Timestamp('2019-02-22 00:00:00')

In [70]:
off_data_before_20190222 = off_data[off_data['read_dt'] < a]
off_data_after_20190222 = off_data[off_data['read_dt'] >= a]

In [99]:
off_data_author_user = pd.concat([off_data_before_20190222['user_id'],off_data_before_20190222['article_id']],axis = 1)
off_data_author_user_after = pd.concat([off_data_after_20190222['user_id'],off_data_after_20190222['article_id']],axis = 1)

In [100]:
len(off_data_author_user)
len(off_data_before_20190222)

19826181

In [104]:
from tqdm import tqdm

In [111]:
dev_read_article_list = []
for i in tqdm(range(3000),desc='progress',mininterval=100):
    read_article = off_data_author_user[off_data_author_user['user_id']==dev_user_id[i]]['article_id']
    if(len(read_article) != 0):
        dev_read_article_list.append(list(set(read_article)))
    else :
        dev_read_article_list.append([])
        
dev_read_article_list_after = []
for i in tqdm(range(3000),desc='progress',mininterval=100):
    read_article = off_data_author_user_after[off_data_author_user_after['user_id']==dev_user_id[i]]['article_id']
    if(len(read_article) != 0):
        dev_read_article_list_after.append(list(set(read_article)))
    else :
        dev_read_article_list_after.append([])
print(len(dev_read_article_list_after))
dev_read_article_list_after[:3]

progress: 100%|████████████████████████████████████████████████████████████████████| 3000/3000 [05:36<00:00,  8.93it/s]


3000


[['@minigorae_259', '@kangsunseng_850', '@brunch_151'],
 ['@dotkvk_19',
  '@rihav_22',
  '@eunseongwrite_42',
  '@insightraveler_50',
  '@dotkvk_20'],
 ['@minigorae_258', '@minigorae_254', '@brunch_151']]

In [ ]:
# dev_users_id, dev_visit_cnt, dev_read_cnt, dev_read_article_list, dev_read_article_list_after
user_dict = {'id':dev_user_id,'visit_cnt': dev_visit_day_cnt,'read_cnt':dev_read_cnt,'read_articles':dev_read_article_list,'read_articles_after_20190222':dev_read_article_list_after}
user_df = pd.DataFrame(user_dict)
user_df.to_csv(directory+'user_df.csv',encoding='ms949')

# user_df 읽어오기

In [11]:

user_df = pd.read_csv(directory+'user_df.csv')
user_df = user_df.drop('Unnamed: 0',1)

In [16]:
len(user_df)

3000

In [17]:
low_read_user = user_df[user_df['read_cnt'] <= 2]
normal_user = user_df[user_df['read_cnt'] >= 3]
print(len(low_read_user))
print(len(normal_user))

#low_read_user.to_csv(directory+'low_read_user.csv',encoding='ms949')
#normal_user.to_csv(directory+'normal_user.csv',encoding='ms949')

163
2837


# 1. 신규 유저 
- 전체글 top 50, 신규글 top 50 

In [27]:
total_top_100 = atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False)[:100]
a = 20190222
a = pd.to_datetime(a,format='%Y%m%d')
c = 20901231
c = pd.to_datetime(c,format='%Y%m%d')
article_20190222_20190301 = atc_read_cnt_nn[atc_read_cnt_nn['reg_datetime'] >= a]
article_20190222_20190301 = article_20190222_20190301[article_20190222_20190301['reg_datetime'] != c]
recent_top_100 = article_20190222_20190301.sort_values(by='read_cnt',ascending=False)[:100]
b = 20190301
b = pd.to_datetime(b,format='%Y%m%d')
#article_20190301_20190314 = atc_read_cnt_nn[atc_read_cnt_nn['reg_datetime'] >= b]
#len(article_20190301_20190314)
# 만약 없으면 metadata에서 'id'있는걸로 해서 찾을 것

In [35]:
total_top_100[:2]

,article_id,read_cnt,id,display_url,keyword_list,magazine_id,reg_ts,sub_title,title,author_id,reg_datetime,reg_dt,type,class
63061,@brunch_141,97206,141.0,https://brunch.co.kr/@brunch/141,"[브런치X빨강머리앤, 빨강머리앤, 출판, 작가]",0.0,1.539742e+12,글·그림 작가 파트너 프로젝트,브런치 작가가 함께 빨강머리 앤을 그리고 쓰다.,@brunch,2018-10-17 11:11:22,2018-10-17,개인,5%
63070,@brunch_151,66903,151.0,https://brunch.co.kr/@brunch/151,"[브런치, 시사회, 브런치무비패스]",0.0,1.550466e+12,브런치 무비 패스 5기 작가 모집,"브런치 무비 패스, 영화의 진한 여운을 나누세요",@brunch,2019-02-18 14:00:34,2019-02-18,개인,5%


In [36]:
recent_top_100[:2]

,article_id,read_cnt,id,display_url,keyword_list,magazine_id,reg_ts,sub_title,title,author_id,reg_datetime,reg_dt,type,class
63071,@brunch_152,8336,152.0,https://brunch.co.kr/@brunch/152,"[인터뷰, 작가, 아나운서]",5223.0,1.551244e+12,꿈을 이룬 작가들의 이야기 36,작가 인터뷰 - 임희정 아나운서의 고백,@brunch,2019-02-27 14:00:48,2019-02-27,매거진,5%
390305,@seochogirl_41,2971,41.0,https://brunch.co.kr/@seochogirl/41,"[남편, 신혼일기, 신혼]",0.0,1.551145e+12,내 남편은 비밀 미남,봄 작가의 신혼일기 #1,@seochogirl,2019-02-26 10:35:57,2019-02-26,개인,5%


In [134]:
recent_top_100[90:]

,article_id,read_cnt,id,display_url,keyword_list,magazine_id,reg_ts,sub_title,title,author_id,reg_datetime,reg_dt,type,class
321819,@myolivenote_1175,541,1175.0,https://brunch.co.kr/@myolivenote/1175,"[청소, 기름때, 후드]",25431.0,1.550812e+12,,폐식용유의 변신? 돈 안 드는 레인지후드 청소법,@myolivenote,2019-02-22 14:02:13,2019-02-22,매거진,5%
48477,@binkond_1262,536,1262.0,https://brunch.co.kr/@binkond/1262,"[고양이, 길고양이, 고양이사진]",0.0,1.551057e+12,,한겨울에도 뽀샤시,@binkond,2019-02-25 10:13:28,2019-02-25,개인,5%
386760,@sdain-sygang_138,532,138.0,https://brunch.co.kr/@sdain-sygang/138,"[치아건강, 치아관리, 건강]",0.0,1.551086e+12,40살이 지났다? 치아 관리할 때입니다.,중장년은 꼭! 치아관리 팁,@sdain-sygang,2019-02-25 18:10:48,2019-02-25,개인,5%
439038,@tenbody_1688,527,1688.0,https://brunch.co.kr/@tenbody/1688,"[운동, 다이어트, 허리]",0.0,1.551049e+12,허리 라인 만드는 운동,매일 5분만 참으면 생기는 허리 라인의 놀라운 변화,@tenbody,2019-02-25 08:01:14,2019-02-25,개인,5%
236586,@jordan777_2727,518,2727.0,https://brunch.co.kr/@jordan777/2727,"[금리, 자산축소, 양적완화]",17646.0,1.550809e+12,,美 Fed '긴축 레이스' 2년 만에 끝낸다,@jordan777,2019-02-22 13:15:00,2019-02-22,매거진,5%
28669,@angelaaa_9,515,9.0,https://brunch.co.kr/@angelaaa/9,"[크루즈, 얼루어, 여행]",42822.0,1.551178e+12,로얄캐리비안 크루즈 Allure of the Seas 서부 캐리비안,세계 최대 22만 톤의 크루즈 얼루어 호의 생활,@angelaaa,2019-02-26 19:44:08,2019-02-26,매거진,5%
54205,@book-writer_39,508,39.0,https://brunch.co.kr/@book-writer/39,"[직장생활, 직장, 불가사의]",39989.0,1.551049e+12,,직장생활 최대 불가사의,@book-writer,2019-02-25 07:57:41,2019-02-25,매거진,5%
132982,@englishspeaking_118,499,118.0,https://brunch.co.kr/@englishspeaking/118,"[영어, 스피킹, 영어회화]",27973.0,1.550838e+12,,레벨별 쉐도우 스피킹 활용법,@englishspeaking,2019-02-22 21:14:52,2019-02-22,매거진,5%
202527,@ideaseller_241,496,241.0,https://brunch.co.kr/@ideaseller/241,"[마케팅, 콘텐츠, 온라인]",34297.0,1.551074e+12,,잠재고객을 사로잡는 콘텐츠를 만드는 6가지 방법,@ideaseller,2019-02-25 14:59:25,2019-02-25,매거진,5%
48480,@binkond_1275,494,1275.0,https://brunch.co.kr/@binkond/1275,"[고양이, 길고양이, 시집]",0.0,1.551172e+12,,묘생,@binkond,2019-02-26 18:12:14,2019-02-26,개인,5%


In [136]:
recent_top_100.to_csv(directory+'recent_top_100_0301.csv',encoding='utf-8')

In [18]:
total_top_100 = atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False)[:100]
a = 20190222
a = pd.to_datetime(a,format='%Y%m%d')
article_20190222_20190301 = atc_read_cnt_nn[atc_read_cnt_nn['reg_datetime'] >= a && atc_read_cnt_nn['reg_datetime']!=]
recent_top_100 = article_20190222_20190301.sort_values(by='read_cnt',ascending=False)[:100]
b = 20190301
b = pd.to_datetime(b,format='%Y%m%d')
#article_20190301_20190314 = atc_read_cnt_nn[atc_read_cnt_nn['reg_datetime'] >= b]
len(article_20190301_20190314)
# 만약 없으면 metadata에서 'id'있는걸로 해서 찾을 것

47

In [29]:
print(len(total_top_100))
print(len(recent_top_100))

100
100


In [37]:
article_20190301_20190314 = atc[atc['reg_datetime'] >= b]

In [39]:
len(article_20190301_20190314)
len(article_20190301_20190314[article_20190301_20190314['article_id'].notnull()])

23950

In [ ]:
# 인기 author로 해야지

In [134]:
article_20190301_20190314.tail()

,id,display_url,article_id,keyword_list,magazine_id,reg_ts,sub_title,title,author_id,reg_datetime,reg_dt,type
643040,136,https://brunch.co.kr/@riglobalization/136,@riglobalization_136,"[브랜드, 콘텐츠, 지역]",0,1553548375000,,로컬 콘텐츠 개발 방법론,@riglobalization,2019-03-26 06:12:55,2019-03-26,개인
643050,1219,https://brunch.co.kr/@binkond/1219,@binkond_1219,"[고양이, 길고양이, 고양이사진]",0,1551397125000,,안구정화냥,@binkond,2019-03-01 08:38:45,2019-03-01,개인
643053,8,https://brunch.co.kr/@laraemotion/8,@laraemotion_8,"[커피, 에스프레소, 허영만]",44748,1553509249000,콘파나 걸(Con Panna girl),콘파나,@laraemotion,2019-03-25 19:20:49,2019-03-25,매거진
643088,314,https://brunch.co.kr/@yumileewyky/314,@yumileewyky_314,"[스타트업, 생각, 공간]",0,1552282089000,내가 처음 쓰는 공간에 대한 이야기,생각들이 나를 지켜보던 공간,@yumileewyky,2019-03-11 14:28:09,2019-03-11,개인
643099,24,https://brunch.co.kr/@uxstar/24,@uxstar_24,"[3D, UI, 제스처]",38917,1553502554000,GIS 서비스,3D 지도의 내비게이션 제스처,@uxstar,2019-03-25 17:29:14,2019-03-25,매거진


In [40]:

off_data_popular_author = off_data.groupby('author_id',as_index='author_read')['author_id'].count()


In [71]:
off_data_popular_author_recent = off_data_after_20190222.groupby('author_id',as_index='author_read')['author_id'].count()
popular_author_top_50_recent = off_data_popular_author_recent.sort_values(ascending=False)[:50]
popular_author_top_50_recent_list = []
for i in range(50):
    popular_author_top_50_recent_list.append(str(popular_author_top_50_recent[i:i+1]).split(' ')[0][10:])

In [98]:
popular_author_top_50_recent_list[:12]

['@brunch',
 '@jordan777',
 '@tenbody',
 '@dailylife',
 '@seochogirl',
 '@dotkvk',
 '@mothertive',
 '@binkond',
 '@aemae-human',
 '@yoonjikwon',
 '@hjl0520',
 '@boot0715']

In [74]:
pop_list = [0,0,0,0,0,0,0,0,0,0]
cnt = 0
for x in article_20190301_20190314['author_id']:
    for i in range(10):
        if(x == popular_author_top_50_recent_list[i]):
            pop_list[i] += 1
print(pop_list)

[1, 111, 73, 80, 3, 3, 9, 34, 35, 0]


In [103]:
d = 20190315
d = pd.to_datetime(d,format='%Y%m%d')
pop_list = [0,0,0,0,0,0,0,0,0,0,0,0]
cnt = 0
for x in article_20190301_20190314[article_20190301_20190314['reg_datetime'] < d]['author_id']:
    for i in range(12):
        if(x == popular_author_top_50_recent_list[i]):
            pop_list[i] += 1
print(pop_list)

[1, 52, 28, 36, 1, 0, 4, 17, 23, 0, 2, 2]


In [104]:
i = 0
s = ''
for x in pop_list:
    if(x != 0):
        k = article_20190301_20190314[article_20190301_20190314['author_id'] == popular_author_top_50_recent_list[i]].sort_values('reg_dt')
        s += ' ' + list(k[k['reg_datetime']<d][x-1:x]['article_id'])[0]
    i+=1
print(s)

 @brunch_153 @jordan777_2797 @tenbody_1768 @dailylife_803 @seochogirl_54 @mothertive_81 @binkond_1290 @aemae-human_103 @hjl0520_32 @boot0715_123


In [87]:
d

Timestamp('2019-03-15 00:00:00')

In [101]:
k = article_20190301_20190314[article_20190301_20190314['author_id'] == '@boot0715'].sort_values('reg_dt')
len(k[k['reg_datetime'] < d])

2

In [47]:
popular_author_top_50 = off_data_popular_author.sort_values(ascending=False)[:50]

In [60]:
popular_author_top_50_list = []
for i in range(50):
    popular_author_top_50_list.append(str(popular_author_top_50[i:i+1]).split(' ')[0][10:])

In [62]:
popular_author_top_50_list[:10]

['@brunch',
 '@tenbody',
 '@jordan777',
 '@dailylife',
 '@binkond',
 '@wikitree',
 '@merryseo',
 '@nareun',
 '@roysday',
 '@englishspeaking']

In [57]:
str(popular_author_top_50[3:4]).split(' ')

['author_id\n@dailylife',
 '',
 '',
 '',
 '256239\nName:',
 'author_id,',
 'dtype:',
 'int64']

In [68]:
pop_list = [0,0,0,0,0,0,0,0,0,0]
cnt = 0
for x in article_20190301_20190314['author_id']:
    for i in range(10):
        if(x == popular_author_top_50_list[i]):
            pop_list[i] += 1
print(pop_list)

[1, 73, 111, 80, 34, 30, 1, 8, 6, 11]


In [105]:
recommend_new = '' # user_id 가 있어야겠지
for i in range(40):
    recommend_new = recommend_new + ' ' + list(total_top_100['article_id'][i:i+1])[0]
    recommend_new = recommend_new + ' ' + list(recent_top_100['article_id'][i:i+1])[0]

for i in range(40,50):
    recommend_new = recommend_new + ' ' + list(total_top_100['article_id'][i:i+1])[0]
recommend_new += s # 이 s는 제일 위에 있음

In [114]:
recommend_new

' @brunch_141 @brunch_152 @brunch_151 @seochogirl_41 @brunch_145 @hjl0520_29 @tenbody_1305 @langman_56 @intlovesong_28 @ladybob_33 @dailylife_207 @gh-kim_5 @hyehyodam_19 @sooperbook_4 @steven_179 @thequestbook_7 @brunch_140 @dryjshin_258 @sangheeshyn_66 @jimbeeny_60 @brunch_142 @daljasee_226 @deckey1985_51 @medea0627_76 @conbus_43 @brunch2gc3_12 @sweetannie_145 @moment-yet_161 @dailylife_219 @yoonjikwon_211 @tenbody_1164 @jktk0623_25 @seochogirl_1 @hoonyqqq_84 @brunch_144 @dancingsnail_67 @x-xv_19 @godw3376_9 @honeytip_945 @onec_546 @brunch_147 @brunch3woz_8 @conbus_35 @dotkvk_20 @mightysense_8 @kkangnpo_99 @honeytip_940 @sweetannie_150 @tenbody_1297 @boot0715_119 @shanghaiesther_46 @wonderlandmag_9 @brunch_143 @whitenest_26 @studiocroissant_43 @peregrino97_944 @chofang1_15 @workerhanee_248 @anti-essay_133 @dotkvk_21 @brunch_149 @gallerysh_31 @brunch_148 @dotkvk_19 @brunch_133 @holidaymemories_152 @nareun_143 @donghlim_25 @seochogirl_6 @lonelyplanet_369 @peregrino97_779 @roysday_316 @b

In [115]:
with open(directory+'recommend_new.txt','w') as file:
    file.write(recommend_new)

In [121]:
with open(directory+'recommend_new.txt','r') as file:
    r = file.readlines()
recommend_new = r[0]

In [122]:
recommend_new

' @brunch_141 @brunch_152 @brunch_151 @seochogirl_41 @brunch_145 @hjl0520_29 @tenbody_1305 @langman_56 @intlovesong_28 @ladybob_33 @dailylife_207 @gh-kim_5 @hyehyodam_19 @sooperbook_4 @steven_179 @thequestbook_7 @brunch_140 @dryjshin_258 @sangheeshyn_66 @jimbeeny_60 @brunch_142 @daljasee_226 @deckey1985_51 @medea0627_76 @conbus_43 @brunch2gc3_12 @sweetannie_145 @moment-yet_161 @dailylife_219 @yoonjikwon_211 @tenbody_1164 @jktk0623_25 @seochogirl_1 @hoonyqqq_84 @brunch_144 @dancingsnail_67 @x-xv_19 @godw3376_9 @honeytip_945 @onec_546 @brunch_147 @brunch3woz_8 @conbus_35 @dotkvk_20 @mightysense_8 @kkangnpo_99 @honeytip_940 @sweetannie_150 @tenbody_1297 @boot0715_119 @shanghaiesther_46 @wonderlandmag_9 @brunch_143 @whitenest_26 @studiocroissant_43 @peregrino97_944 @chofang1_15 @workerhanee_248 @anti-essay_133 @dotkvk_21 @brunch_149 @gallerysh_31 @brunch_148 @dotkvk_19 @brunch_133 @holidaymemories_152 @nareun_143 @donghlim_25 @seochogirl_6 @lonelyplanet_369 @peregrino97_779 @roysday_316 @b

In [123]:
# 신규유저, 기존 유저 list 검사
low_read_user = user_df[user_df['read_cnt'] <= 2]
amb_normal_user = user_df[user_df['read_cnt'] <= 5]
normal_user = user_df[user_df['read_cnt'] >5]
print(len(low_read_user))
print(len(amb_normal_user)-len(low_read_user))
print(len(normal_user))

163
94
2743


In [112]:
amb_normal_user

,id,visit_cnt,read_cnt,read_articles,read_articles_after_20190222
3,#0085e940b6e94be6efe4e275d1966615,3,7,"['@hotelscomkr_617', '@prestigegorilla_349', '...",[]
6,#009bca89575df8ed68a302c1ceaf7da4,1,4,[],"['@moripark_84', '@moripark_79', '@moripark_82..."
17,#0167896db82acc4fd94275913af19a04,2,2,['@sakiroo_11'],['@brunch_151']
26,#0200c6c68d23cb49a8a06afa7e6b7e25,1,4,[],"['@wikitree_3228', '@hitchwill_2070']"
35,#028bdbaae99a474017f7ce0a1d531713,1,1,[],['@spainmusa_24']
64,#053f46f46077943720b23934e095c1f6,2,3,[],"['@prestigegorilla_368', '@allstay_1167', '@ne..."
70,#058b56d54a2987a8fc22abbf465d2274,1,1,[],['@kimchohee_2']
76,#0666a44447d7c6835c726e15bceffd02,1,1,[],['@aemae-human_66']
93,#07de1cc9328086ecce32f97fc35f7bbb,5,9,[],"['@jhzflk_22', '@hoonyqqq_84', '@seokjipkim_2'..."
96,#08a602b9a81f5cfdb99ac229d41f9be1,3,6,"['@sooperbook_3', '@icallu_110', '@dailylife_2...","['@paulie_88', '@whizzer4_64', '@dailylife_207']"


In [ ]:
#  참고자료
- 파일 입출력 : https://wikidocs.net/14593
- EDA : https://arena.kakao.com/forum/topics/10
- list to pandas : https://pbpython.com/pandas-list-dict.html
        -https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-lists/
- pandas to csv :  https://mjdeeplearning.tistory.com/41
- csv to pandas dataframe : https://godoftyping.wordpress.com/2018/08/15/python-pandas%EC%97%90%EC%84%9C-csv-%ED%8C%8C%EC%9D%BC-%EC%9D%BD%EA%B8%B0-%EC%93%B0%EA%B8%B0/
- tqdm : https://pypi.org/project/tqdm/
        -https://github.com/tqdm/tqdm